In [ ]:
#imports
import cv2
import numpy as np

In [ ]:
#load in the videos

In [2]:
# Function that is able to detect face of frame in video

In [3]:
# Detect Speed of face in 2D video

In [ ]:
# Detect speed of face using depth data
# This will be tied to the distance of nose